In [1]:
# Materials
YoungModulus = {  # Pa, Pascals
    'steel' : 2.1e11,
    'nylon.min' : 1.59e9,
    'nylon.max' : 3.79e9,
    'nylon.avg' : 2.69e9
}

# Available sections
EffectiveBarRadius = {  # m, meters
    'M4' : 1.5e-3,
    'M5' : 2.0e-3,
    'M6' : 2.5e-3,
    'M8' : 3.5e-3
}

In [ ]:
import numpy as np

# Single storey definition
n_cols = 4
storey_h = 0.22            # m
storey_m = 0.14 + 0.230    # kg   Storey masses : 0.2  0.14 | Additional masses : 0.23  0.48
r = EffectiveBarRadius['M6']
E = YoungModulus['nylon.avg']
I = np.pi*r**4/4
storey_k = n_cols*12*E*I/storey_h**3
omega = np.sqrt(storey_k/storey_m)
print(f'Frequency : {omega/2/np.pi} Hz')

Frequency : 3.169269011746345 Hz


In [3]:
# Matrix system for a multi-storey building
n_storeys = 3
m = storey_m * np.ones(n_storeys)
k = storey_k * np.ones(n_storeys)
damping = 0.05

print(f'Total mass   : {storey_m * n_storeys:.1f} kg')
print(f'Total height : {storey_h * n_storeys:.2f} m')

# Mass and stiffness matrices
M = np.zeros((n_storeys, n_storeys))
K = np.zeros((n_storeys, n_storeys))
M[0,0] = m[0]
K[0,0] = k[0]
for i in range(1,n_storeys):
  M[i,i] += m[i]
  K[i,i] += k[i]
  K[i-1,i-1] += k[i]
  K[i,i-1] += -k[i]
  K[i-1,i] += -k[i]

Total mass   : 1.1 kg
Total height : 0.90 m


In [4]:
from scipy.linalg import eigh
eigvals, eigvecs = eigh(K, M)
eigvecs_n = [v/np.sqrt(v@M@v) for v in eigvecs.T] # normalization
eigvecs_n = np.array(eigvecs_n)      # casting into numpy array

In [5]:
frequencies = eigvals**0.5
periods = 2*np.pi/frequencies
print('Periods      : [' + ', '.join([f'{p:.2f}' for p in periods]) + '] s')
print('Natural freq : [' + ', '.join([f'{w/2/np.pi:.2f}' for w in frequencies]) + '] Hz')
print(f'Pendulum length : {9.81/frequencies[0]**2*100:.2f} cm')

Periods      : [0.71, 0.25, 0.18] s
Natural freq : [1.41, 3.95, 5.71] Hz
Pendulum length : 12.49 cm


In [6]:
d = np.ones(n_storeys)
modal_mass = [(v@M@v) / (v@v) for v in eigvecs_n] # eigvecs.T
participation = [(v@M@d) / (v@M@v) for v in eigvecs_n] # eigvecs.T
eff_mass = [(v@M@d)**2 / (v@M@v) for v in eigvecs_n] # eigvecs.T
eff_mass_rel = eff_mass / np.trace(M)
print(f'Total mass      : {sum(eff_mass)}')
print('Effective  mass : ' + ', '.join([f'{m:.3f}' for m in eff_mass]))
print('                  ' + ', '.join([f'{m:.1%}' for m in eff_mass_rel]))

Total mass      : 1.11
Effective  mass : 1.015, 0.083, 0.012
                  91.4%, 7.5%, 1.1%


In [7]:
g = 9.81            # m/s^2
dt = 0.02           # the time step considered by the input data
seism_scale = 0.01  # convert dm/s^2 to m/s^2
seism = np.loadtxt('AcMx1985.txt') * seism_scale
times = np.arange(len(seism))*dt

In [8]:
from scipy.integrate import solve_ivp

# NOTE: this method is not optimized because of the interpolation.
# Therefore, it is slower, although more accurate
def SystemDerivatives(t, y, frequency, damping):
    u = y[0]
    v = y[1]
    s = np.interp(t, times, seism)
    a = s - 2*damping*frequency*v - frequency**2*u
    return [v, a]

def MaxResponseInt(period, damping):
    frequency = 2*np.pi/period
    solution = solve_ivp(fun=SystemDerivatives, t_span=[0, times[-1]],
                         y0=[0,0], t_eval=times, args=[frequency, damping])
    a = frequency**2*solution.y[0]
    return max(np.absolute(a))

Sa = [MaxResponseInt(T,damping) for T in periods]
print('Sa : ', Sa)

Sa :  [np.float64(0.9711853411281505), np.float64(1.762773101637254), np.float64(1.7402758826352103)]


In [9]:
u_max = []
v_max = []
a_max = []

for i in range(n_storeys):
    w = frequencies[i]
    u_max.append(participation[i]*Sa[i]/w/w*eigvecs_n[i])
    v_max.append(participation[i]*Sa[i]/w*eigvecs_n[i])
    a_max.append(participation[i]*Sa[i]*eigvecs_n[i])

## Quadratic combination

$u_{CQC} = \sqrt{\sum_i u_i^2}$

In [10]:
print(u_max[0])

[0.00671629 0.01210234 0.01509137]


In [11]:
u_cqc = 0
for u_i in u_max:
    u_cqc += np.power(u_i,2)
u_cqc = np.sqrt(u_cqc)
u_cqc_rel = np.array(u_cqc) # creating a copy of the array
for i in range(1,n_storeys):
    u_cqc_rel[i] = u_cqc[i] - u_cqc[i-1]
print(f'Complete quadratic combination')
print(f'absolute displacements : {u_cqc}')
print(f'relative displacements : {u_cqc_rel}')
print(3/500)

Complete quadratic combination
absolute displacements : [0.00679168 0.01211185 0.01511281]
relative displacements : [0.00679168 0.00532017 0.00300096]
0.006


## Base shear and overturning moment

$V_b = mS_a(\omega_1)$

$M_b = V_b \frac{\sum_i m_i z_i^2}{\sum_i m_i z_i}$

In [12]:
#Don't need. TO BE REMOVED
dist_h = 8   # m
A = np.pi * col_diam**2 / 4   # m^2
W = 2e-4     # m^3

NameError: name 'col_diam' is not defined

In [ ]:
z = np.arange(storey_h, storey_h*n_storeys, storey_h)
mass = np.trace(M)
Vb = mass*Sa[0]
print(m.shape)
print(z.shape)
Mb = Vb * np.dot(m, np.power(z,2)) / np.dot(m,z)
print(Vb)
print(Mb)

## Structure verification

In this example, we assume all the forces are equally distributed on all the columns. Each column must verify the following actions:

Shear: $V_{ED} = \frac{V_b}{n_{columns}}$

Bending moment: $M_{ED} = \frac{V_{ED}h}{2}$

Axial force: $N_{ED} = \frac{mg}{n_{columns}} + \frac{2M_b}{n_{columns}h_{dist}}$

In [ ]:
Ved = Vb / n_cols
Med = Ved * storey_h / 2
Ned = mass*10 / n_cols + 2*Mb/n_cols/dist_h
print(f'{Ved = } N')
print(f'{Med = } Nm')
print(f'{Ned = } N')

Resisting properties of a steel section

$V_{RD} = f_yA/\sqrt{3}$

$M_{RD} = f_yW$

$N_{RD} = f_yA$

In [ ]:
fy = 3.6e8 # Pa
Vrd = fy*A/np.sqrt(3)
Mrd = fy*W
Nrd = fy*A
print(Vrd)
print(Mrd)
print(Nrd)

Verification

In [ ]:
print(f'Ratio V : {Ved/Vrd}')
print(f'Ratio M : {Med/Mrd}')
print(f'Ratio N : {Ned/Nrd}')